In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load dataset

In [ ]:
os.chdir("c:/Users/victo/Personal_Projects/spriometry/")
os.getcwd()

In [ ]:
demographics = pd.read_csv('spirometry_data/demographics_data.csv')
spirometry = pd.read_csv('spirometry_data/spirotidydatafinal.txt', sep=',')

In [ ]:
demographics.head()

In [ ]:
# spirometry.dropna(inplace=True)
spirometry.head()

In [ ]:
spirometry[(spirometry['ID'] == 102) & (spirometry['Visit'] == 1)]["Trial"].unique()

In [ ]:
id101_visit1_trial1 = spirometry[(spirometry['ID'] == 101) & (spirometry['Visit'] == 1) & (spirometry['Trial'] == 4)]
# id101_visit1_trial1

# Restructuring the data

In [ ]:
spirometry[spirometry['ID'] == 101]

In [ ]:
spirometry["ID"].unique()

In [83]:
patient_ids = spirometry['ID'].unique()

for patient_id in patient_ids:
    # Create a directory for each patient
    # os.mkdir(f'spirometry_data/ID{patient_id}')
    # os.chdir(f'spirometry_data/ID{patient_id}')
    
    # Get the data for each patient
    patient_data = spirometry[spirometry['ID'] == patient_id]

    accepted_flow, accepted_volume = pd.DataFrame(), pd.DataFrame()
    trials_array, visits_array = np.array([]), np.array([])
    
    # Get the number of visits each patient
    n_visits = patient_data['Visit'].unique()    
    for visit in n_visits:
        visit_data = patient_data[patient_data['Visit'] == visit]
        n_trials = patient_data[patient_data['Visit'] == visit]['Trial'].unique()
        for trial in n_trials:
            visits_array = np.append(visits_array, visit)
            trials_array = np.append(trials_array, trial)
            
            # Creating Volume and Flow arrays for each trial
            vol_trex = visit_data[visit_data['Trial'] == trial]['Volume']
            flow_trex = visit_data[visit_data['Trial'] == trial]['Flow']
            accepted_flow = pd.concat([accepted_flow, flow_trex.reset_index(drop=True)], axis=1)
            accepted_volume = pd.concat([accepted_volume, vol_trex.reset_index(drop=True)], axis=1)
            
    accepted_flow = accepted_flow.T
    accepted_volume = accepted_volume.T
    subplot_size = np.sqrt(len(n_trials))
    m = 0
    AIN = np.array([])

    visits_array = visits_array.astype(int)
    trials_array = trials_array.astype(int)

    #  Loop throguh the trials to calculate AIN
    for i, trial in enumerate(trials_array):

        vol_ain = accepted_volume.iloc[i].to_numpy().reshape(1, -1)
        flow_ain = accepted_flow.iloc[i].to_numpy().reshape(1, -1)
        m += 1
        # Finding peak of inspiratory (since inspiratory flow is negative, 
        # we use the minimum value of flow as the peak of inspiratory flow)
        min_flow, min_flow_idx = np.nanmin(flow_ain, axis=1), np.nanargmin(flow_ain, axis=1)[0]
        
        # Finding the maximum volume and its index
        max_vol, max_vol_idx = np.nanmax(vol_ain), np.nanargmax(vol_ain, axis=1)[0]
        # Find the 2nd index for ain calc (dont really understand this part)
        ain_val2_idx = np.where(flow_ain[0][min_flow_idx:] >= 0)[0]
        ain_val2 = flow_ain[min_flow_idx:][ain_val2_idx]
        print()

(1849,)
834
[ 47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64
  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82
  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100
 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154
 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172
 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190
 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208
 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226
 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241]


IndexError: index 47 is out of bounds for axis 0 with size 0

In [82]:
import numpy as np

# Sample numpy array
np.random.seed(42)
arr = np.random.randint(0, 100, size=(1, 10))
print(np.nanargmax(arr, axis=1))
# Get indices of elements greater than 0
indices = np.where(arr[0][2:] >= 0)[0]

print("Indices of elements greater than 0:", indices)


[1]
Indices of elements greater than 0: [0 1 2 3 4 5 6 7]


In [ ]:

    #     # Find the 2nd index for ain calc (dont really understand this part)
    #     ain_val2_idx = np.where(flow_ain[min_flow_idx:] >= 0)[0]
    #     ain_val2 = flow_ain[min_flow_idx:][ain_val2_idx]
    #     print(ain_val2_idx, ain_val2)
    #     # Acceptence criteria: Inspiratory Flow must be greater than 1
    #     # L/s and Maximum volume must be greater than 0.5 L in order to
    #     # proceed with AIN calculation.
    #     if abs(min_flow) > 1 and max_vol > 0.5:
            
    #         # Ensuring inspiratory curve is where data first enters
    #         # negative flow up until it first passes into positive flow.
    #         if not ain_val2 and not ain_val2_idx:
    #             # Acceptance criteria:
    #             # Inspiratory flow must be as close as 0.5 L/s away from 0
    #             # to proceed with calculation
    #             if min(abs(flow_ain[min_flow_idx:])) > 0.5:
    #                 ain_val2 = np.NaN
    #                 ain_val2_idx = np.NaN
    #                 disq = 1
    #             else:
    #                 ain_val2_idx = np.where(np.isnan(flow_ain[min_flow_idx:]))[0]
    #                 ain_val2 = flow_ain[min_flow_idx:][ain_val2_idx]
    #         else:
    #             pass